In [55]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the CSV file


df1, df2, df3 = pd.read_csv(text_1_path), pd.read_csv(text_2_path), pd.read_csv(text_3_path)
df4, df5, df6 = pd.read_csv(text_4_path), pd.read_csv(text_5_path), pd.read_csv(text_6_path)

dataprev = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
data = pd.read_csv("csv files/all_files.csv")
# Ensure the necessary column exists
if "text" not in data.columns:
    raise ValueError("The column 'preprocessed_text' is missing from the CSV file.")
    
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data['text'])

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix)

In [66]:
thresholdLowerBound = 0.98  # Adjust this as needed
thresholdUpperBound = 1 # Adjust this as needed

# Initialize a set to keep selected files
selected_files = set()

# Keep track of files already represented
represented_files = set()

# Sort similar pairs to ensure consistent results
similar_pairs_sorted = sorted(similar_pairs, key=lambda x: -x[2])  # Sort by similarity score descending

for file1, file2, score in similar_pairs_sorted:
        # If neither file is already represented, add one of them
    if file1 not in represented_files and file2 not in represented_files:
        selected_files.add(file1)  # Arbitrarily choose file1
    represented_files.add(file1)
    represented_files.add(file2)    

# Display results
uniqueFiles = set(file for file in fileNames if file not in represented_files)
print(len(represented_files), f"files have >= 1 duplicate at threshold: {thresholdLowerBound}")
duplicatesRemoved = selected_files.union(uniqueFiles)
filesToRemove = set(file for file in fileNames if file not in duplicatesRemoved)

assert duplicate_files.issubset(represented_files), "Not all duplicates are represented!"
assert selected_files.issubset(represented_files), "Selected files are not correctly represented!"

print(f"Number of selected files: {len(selected_files)} out of {len(similar_pairs_sorted)} pairs.\n"+
      f"Number of files once duplicates were removed: {len(duplicatesRemoved)}.\n"+
      f"Originally {len(data)} files.")


320 files have >= 1 duplicate at threshold: 0.98
Number of selected files: 77 out of 10209 pairs.
Number of files once duplicates were removed: 6607.
Originally 6850 files.


In [70]:
similar_pairs = [] # store file pairs with similarity above the threshold

mask = (similarity_matrix > thresholdLowerBound) & (similarity_matrix < thresholdUpperBound) # Mask similarity matrix to find pairs above threshold
indices = np.argwhere(mask)
fileNames = data["file_name"].to_numpy()

similar_pairs = [(fileNames[i], fileNames[j], similarity_matrix[i, j]) # Collect file names for pairs
                 for i, j in indices if i < j]  # Only upper triangle, since similarityMatrix is symmetrical (i.e. sm[i,j] = sm[j,i])

duplicate_files = set(file1 for file1, file2, _ in similar_pairs).union(file2 for file1, file2, _ in similar_pairs)
print(f"There are {len(duplicate_files)} files with a duplicate. There are {len(fileNames) -len(duplicate_files)} unique files.\n")

# Display the results
print(f"There are {len(similar_pairs)} pairs of files with similarity above {thresholdLowerBound} and below {thresholdUpperBound} out of {len(data)} total files:\n")

uniqueFiles = set(file for file in fileNames if file not in duplicate_files)
print(len(uniqueFiles))

There are 320 files with a duplicate. There are 6530 unique files.

There are 10209 pairs of files with similarity above 0.98 and below 1 out of 6850 total files:

6530


### Below is the final dataset with one copy of each duplicate

In [76]:
dataNoDuplicates = data[~data['file_name'].isin(filesToRemove)].reset_index(drop=True)
dataNoDuplicates.to_csv('uniqueDataset.csv', index=False)